In [1]:
# pip install pandas torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [3]:
import os
import pandas as pd
import numpy as np
import torch

print(torch.__version__)
print(f"torch.cuda.is_available: {torch.cuda.is_available()}")
print(f"torch.version.cuda: {torch.version.cuda}\n")


DATA_PATH = "./data/NIFTY 100_minute_data.csv"

if not os.path.exists(DATA_PATH):
	print("Download data from https://www.kaggle.com/datasets/debashis74017/nifty-50-minute-data")
else:
	print("Loading data...")
	data = pd.read_csv(DATA_PATH)
	print("Loading data - SUCCESS.")

2.7.0+cu128
torch.cuda.is_available: True
torch.version.cuda: 12.8

Download data from https://www.kaggle.com/datasets/debashis74017/nifty-50-minute-data


In [3]:
data.shape

(932522, 6)

In [4]:
cols_to_use = ["close"]
data = data[["date"]+ cols_to_use]
data

,date,close
0,2015-01-09 09:15:00,8308.35
1,2015-01-09 09:16:00,8304.20
2,2015-01-09 09:17:00,8308.20
3,2015-01-09 09:18:00,8315.30
4,2015-01-09 09:19:00,8316.10
...,...,...
932517,2025-02-07 15:25:00,24130.35
932518,2025-02-07 15:26:00,24129.60
932519,2025-02-07 15:27:00,24137.35
932520,2025-02-07 15:28:00,24130.80


In [5]:
lags = [1,2,3,5, 20, 60]

assert(len(lags) == 6)
for lag in lags:
   data[f"lag_{lag}"] = data["close"].shift(lag)

data = data.dropna()
data

,date,close,lag_1,lag_2,lag_3,lag_5,lag_20,lag_60
60,2015-01-09 10:15:00,8296.25,8294.75,8292.25,8290.50,8292.60,8290.65,8308.35
61,2015-01-09 10:16:00,8294.50,8296.25,8294.75,8292.25,8288.60,8293.00,8304.20
62,2015-01-09 10:17:00,8295.65,8294.50,8296.25,8294.75,8290.50,8296.70,8308.20
63,2015-01-09 10:18:00,8295.60,8295.65,8294.50,8296.25,8292.25,8293.45,8315.30
64,2015-01-09 10:19:00,8297.35,8295.60,8295.65,8294.50,8294.75,8296.05,8316.10
...,...,...,...,...,...,...,...,...
932517,2025-02-07 15:25:00,24130.35,24131.15,24131.35,24134.85,24133.65,24090.35,24017.75
932518,2025-02-07 15:26:00,24129.60,24130.35,24131.15,24131.35,24138.25,24101.70,24016.30
932519,2025-02-07 15:27:00,24137.35,24129.60,24130.35,24131.15,24134.85,24104.80,24019.85
932520,2025-02-07 15:28:00,24130.80,24137.35,24129.60,24130.35,24131.35,24111.35,24014.45


In [6]:
filename = "NIFTY100_1m"
data.to_csv(f'./ETDataset/ETT-small/{filename}.csv', index=False)

# NEW DATASET

In [4]:
DATA_PATH = "GAZP_combo.csv"

data = pd.read_csv(DATA_PATH, delimiter=";")
initial_nrows = data.shape[0]
data.shape

(63738, 1)

In [5]:
data["date"] = pd.to_datetime(data["<DATE>"].combine(data["<TIME>"], lambda x, y: str(x)+" "+str(y)))
data["date"]
data

KeyError: '<DATE>'

In [73]:
data = data.drop(["<DATE>", "<TIME>"], axis=1)
data.dropna(inplace=True)
print(data.shape)
# assert(initial_nrows == data.shape[0])

(63738, 7)


In [74]:
data = data[['date', 'bz', 'eurrub', 'moex', 'rgbi', 'usdrub', 'gazp']]
data

,date,bz,eurrub,moex,rgbi,usdrub,gazp
0,2024-01-03 10:00:00,77.08,98.501,3100.91,121.79,89.3510,159.94
1,2024-01-03 10:01:00,77.08,98.501,3102.54,121.81,89.3510,159.91
2,2024-01-03 10:02:00,77.08,98.501,3104.13,121.82,90.4750,159.95
3,2024-01-03 10:03:00,77.08,98.501,3106.06,121.76,90.5174,160.04
4,2024-01-03 10:04:00,77.08,98.501,3107.47,121.72,90.5514,160.21
...,...,...,...,...,...,...,...
63733,2024-10-31 19:08:00,73.40,113.962,2678.71,100.11,101.9760,123.06
63734,2024-10-31 19:09:00,73.41,113.996,2677.78,100.11,102.0500,123.06
63735,2024-10-31 19:10:00,73.41,113.981,2677.72,100.11,102.2260,123.07
63736,2024-10-31 19:11:00,73.41,113.945,2677.86,100.10,102.3108,123.05


In [75]:
filename = "GAZP_combo"
data.to_csv(f'./ETDataset/ETT-small/{filename}.csv', index=False)

# NEW DATASET

In [34]:
dfs_list = []
for file in ["mar23.csv", "jun23.csv", "sep23.csv", "dec23.csv"]:
	data = pd.read_csv(file, delimiter=";")
	data["date"] = pd.to_datetime(data["<DATE>"].combine(data["<TIME>"], lambda x, y: str(x)+" "+str(y)))
	data["date"]
	print(data.shape)
	data = data.drop(["<DATE>", "<TIME>"], axis=1)
	data.dropna(inplace=True)
	dfs_list.append(data)

GAZP_df = pd.read_csv("GAZP_combo.csv", delimiter=",")
GAZP_df["date"] = pd.to_datetime(GAZP_df["date"])
print(GAZP_df.shape)
dfs_list.append(GAZP_df)

result = pd.concat(dfs_list, ignore_index=True, axis=0)
result = result[['date', 'bz', 'eurrub', 'moex', 'rgbi', 'usdrub', 'gazp']]
result.sort_values(by="date", ascending=True)
print(result.shape)
result

(10093, 9)
(12985, 9)
(13377, 9)
(13844, 9)
(63738, 7)
(114037, 7)


,date,bz,eurrub,moex,rgbi,usdrub,gazp
0,2023-01-03 10:00:00,85.91,78.932,2160.26,130.78,71.0007,162.97
1,2023-01-03 10:01:00,85.55,78.932,2157.89,130.77,72.6716,162.51
2,2023-01-03 10:02:00,85.64,78.695,2158.49,130.76,71.5000,162.66
3,2023-01-03 10:03:00,85.84,78.697,2158.80,130.84,72.4878,162.44
4,2023-01-03 10:04:00,85.88,78.675,2159.81,130.81,69.9112,162.40
...,...,...,...,...,...,...,...
114032,2024-10-31 19:08:00,73.40,113.962,2678.71,100.11,101.9760,123.06
114033,2024-10-31 19:09:00,73.41,113.996,2677.78,100.11,102.0500,123.06
114034,2024-10-31 19:10:00,73.41,113.981,2677.72,100.11,102.2260,123.07
114035,2024-10-31 19:11:00,73.41,113.945,2677.86,100.10,102.3108,123.05


In [35]:
result.to_csv("final_df_114037.csv", index=False)